# AoC 2024 - Day 13

<https://adventofcode.com/2024/day/13>

In [1]:
# Parameters
# 
day = 13

FILENAME = "day13_test1.txt"
#
FILENAME = "day13_input_gmacario.txt"

In [2]:
# Import
from icecream import ic
import time

In [3]:
# machine_0_config = {
#     "a": (94, 34),
#     "b": (22, 67),
#     "p": (8400, 5400)
# }

# ic(machine_0_config)

In [4]:
# Read the puzzle input into a data structure, one per line

def s2i(s) -> int:
    return int(s.split(",")[0])

machines = list()
num_machines = 0
with open(FILENAME, 'r') as f:
    again = True
    while again:
        _, a_x, a_y = f.readline().split("+")
        _, b_x, b_y = f.readline().split("+")
        _, p_x, p_y = f.readline().split("=")
        entry = {
            "A": (s2i(a_x), s2i(a_y)),
            "B": (s2i(b_x), s2i(b_y)),
            "P": (s2i(p_x), s2i(p_y))
        }
        # ic(entry)
        machines.append(entry)
        num_machines += 1
        if f.readline() == "":
            again = False

# _ = ic(num_machines, machines)
tm_begin = time.time()
ic(time.ctime())

ic| time.ctime(): 'Sat Dec 21 11:53:46 2024'


'Sat Dec 21 11:53:46 2024'

## Part 1

In [5]:
# # Try some movement on machines[0]

# mach = machines[0]
# ic(mach)
# pos = (0, 0)
# cost = 0

# # push("A")
# cost += 3
# new_x = pos[0] + mach['A'][0]
# new_y = pos[1] + mach['A'][1]
# pos = (new_x, new_y)
# _ = ic("After push(A):", cost, pos)

# # push("B")
# cost += 1
# new_x = pos[0] + mach['B'][0]
# new_y = pos[1] + mach['B'][1]
# pos = (new_x, new_y)
# _ = ic("After push(B):", cost, pos)

In [6]:
def check_winning_sequence(mach: dict, initial_pos: tuple, num_a: int, num_b: int) -> int:
    """Find if some movement are winning
    Return the cost of movement, or -1 if it does not win
    """
    # ic(mach, initial_pos, num_a, num_b)

    # if num_a > 100 or num_b > 100:
    #     # ic(num_a, num_b, "too big")
    #     return -1

    end_x = initial_pos[0]
    end_y = initial_pos[1]
    cost = 0

    end_x = end_x + mach['A'][0] * num_a
    end_y = end_y + mach['A'][1] * num_a
    cost += num_a * 3

    end_x = end_x + mach['B'][0] * num_b
    end_y = end_y + mach['B'][1] * num_b
    cost += num_b * 1

    if (end_x, end_y) == (mach['P'][0], mach['P'][1]):
        return cost
    else:
        # ic((end_x, end_y), "not in", mach['P'])
        return -1

In [7]:
# assert check_winning_sequence(machines[0], (0,0), 80, 40) == 280
# assert check_winning_sequence(machines[2], (0,0), 38, 86) == 200

In [8]:
def find_cheapest_winning_sequence(mach: dict, initial_pos, max_steps:int=100) -> tuple:
    """
    pos: tuple for starting position
    Return a tuple (best_num_a:int, best_num_b:int, best_cost:int)
    """
    # ic("find_cheapest_winning_sequence:", mach, initial_pos)
    best_num_a = None
    best_num_b = None
    best_cost = None
    for num_a in range(1, max_steps + 1):
        for num_b in range(1, max_steps + 1):
            # ic("Trying", num_a, num_b)
            c = check_winning_sequence(mach, initial_pos, num_a, num_b)
            if c < 0:
                pass
            elif best_cost is None and c > 0:
                best_num_a = num_a
                best_num_b = num_b
                best_cost = c
                ic("Found first", best_cost, "with", best_num_a, best_num_b)
            elif best_cost is not None and c < best_cost:
                best_num_a = num_a
                best_num_b = num_b
                best_cost = c
                ic("Found better", best_cost, "with", best_num_a, best_num_b)

    return (best_num_a, best_num_b, best_cost)

# _ = ic(machines)

_ = ic(find_cheapest_winning_sequence(machines[0], (0,0))) # (80, 40, 280)
# _ = ic(find_cheapest_winning_sequence(machines[1], (0,0))) # No combination
# _ = ic(find_cheapest_winning_sequence(machines[2], (0,0))) # (38, 86, 200)
# _ = ic(find_cheapest_winning_sequence(machines[3], (0,0))) # No combination

# assert find_cheapest_winning_sequence(machines[0], (0,0)) == (80, 40, 280)

ic| find_cheapest_winning_sequence(machines[0], (0,0)): (None, None, None)


In [9]:
def find_winning_sequence_linalgebra(mach):
    # ic(mach)
    p_x = mach['P'][0]
    p_y = mach['P'][1]
    a_dx = mach['A'][0]
    a_dy = mach['A'][1]
    b_dx = mach['B'][0]
    b_dy = mach['B'][1]

    n_a_num = p_x * b_dy - p_y * b_dx
    n_a_den = a_dx * b_dy - a_dy * b_dx
    n_a = n_a_num // n_a_den

    if n_a_num != n_a * n_a_den:
        return 0, 0, None

    n_b_num = p_x * a_dy - p_y * a_dx
    n_b_den = b_dx * a_dy - b_dy * a_dx
    n_b = n_b_num // n_b_den

    if n_b_num != n_b * n_b_den:
        return 0, 0, None

    result = n_a, n_b, n_a * 3 + n_b * 1

    # ic("find_winning_sequence", mach, ": found", result)
    return result

In [10]:
# total_tokens = 0
# for mach in machines:
#     n_a, n_b, n_tokens = find_cheapest_winning_sequence(mach)
#     ic(n_a, n_b, n_tokens)
#     if n_tokens is not None:
#         total_tokens += n_tokens

# _ = ic(total_tokens)

def sum_cheapest_sequences(machines, initial_pos:tuple=(0,0), max_steps:int=100):
    
    result = 0
    for mach in machines:
        # n_a, n_b, n_tokens = find_cheapest_winning_sequence(mach, initial_pos)
        n_a, n_b, n_tokens = find_winning_sequence_linalgebra(mach)
        #
        # ic(n_a, n_b, n_tokens)
        if n_tokens is not None:
            result += n_tokens
    return result

In [11]:
def solve_part1():
    return sum_cheapest_sequences(machines)

# tm_begin = time.time()
part1_result = solve_part1()
tm_end = time.time()
ic("elapsed=", tm_end - tm_begin, "result=", part1_result)
print("Day 13 Part 1 RESULT:")
print(part1_result)

ic| "elapsed=": 'elapsed='
    tm_end - tm_begin: 0.18650245666503906
    "result=": 'result='
    part1_result: 37680


Day 13 Part 1 RESULT:
37680


## Part 2

In [12]:
import copy

machines_corrected = copy.deepcopy(machines)
initial_pos = (0, 0)

for mach in machines_corrected:
    new_p_x = mach['P'][0] + 10_000_000_000_000
    new_p_y = mach['P'][1] + 10_000_000_000_000
    mach['P'] = (new_p_x, new_p_y)

# _ = ic(machines)
# _ = ic(machines_corrected)

_ = ic(find_winning_sequence_linalgebra(machines_corrected[0]))
_ = ic(find_winning_sequence_linalgebra(machines_corrected[1]))
_ = ic(find_winning_sequence_linalgebra(machines_corrected[2]))
_ = ic(find_winning_sequence_linalgebra(machines_corrected[3]))


ic| find_winning_sequence_linalgebra(machines_corrected[0]): (182025028915, 102389078453, 648464165198)
ic| find_winning_sequence_linalgebra(machines_corrected[1]): (0, 0, None)
ic| find_winning_sequence_linalgebra(machines_corrected[2]): (0, 0, None)
ic| find_winning_sequence_linalgebra(machines_corrected[3]): (0, 0, None)


In [13]:
def solve_part2():
    return sum_cheapest_sequences(machines_corrected, initial_pos)

# tm_begin = time.time()
part2_result = solve_part2()
tm_end = time.time()
ic("elapsed=", tm_end - tm_begin, "result=", part2_result)
print("Day 13 Part 2 RESULT:")
print(part2_result)

ic| "elapsed=": 'elapsed='
    tm_end - tm_begin: 0.31586480140686035
    "result=": 'result='
    part2_result: 87550094242995


Day 13 Part 2 RESULT:
87550094242995
